In [3]:
!pip install pymorphy2
!pip install jsonlines

In [41]:
from sklearn.metrics import mean_squared_error
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pymorphy2
import pandas as pd
import pickle
import jsonlines
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.metrics.pairwise import linear_kernel
from gensim.models import word2vec
import numpy as np
morph = pymorphy2.MorphAnalyzer()

stops = set(stopwords.words("english")) | set(stopwords.words("russian"))
def review_to_wordlist(review):
    #1)
    review_text = re.sub("[^а-яА-Яa-zA-Z1-9]"," ", review)
    #2)
    words = review_text.lower().split()
    #3)
    words = [w for w in words if not w in stops]
    #4)
    words = [morph.parse(w)[0].normal_form for w in words ]
    return(words)

def clear_text(review):
    #1)
    review_text = re.sub("[^а-яА-Яa-zA-Z1-9]"," ", review)
    #2)
    words = review_text.split()
    #3)
    #words = [w for w in words if not w in stops]
    #4)
    #words = [morph.parse(w)[0].normal_form for w in words ]
    return(" ".join(words))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [102]:
data = pd.read_csv("/content/drive/MyDrive/commondata_form_data (5).csv")

In [103]:
data.head()

,question,type,status
0,Отчетство,ShortText,1
1,Сколько ву вас сотрудников?,ShortText,0
2,"Часто ли вы планируете, как будете себя вести ...",OneChoice,0
3,Какого литературного героя вы хотели бы встрет...,ShortText,0
4,Вам легко найти общий язык с незнакомыми людьми?,OneChoice,0


In [76]:
data["type"].unique()

array(['OneChoice', 'ShortText', 'MultipleChoice'], dtype=object)

In [10]:
#ShortText - 0
#OneChoice - 1
#MultipleChoice - 2
def type_to_encode(type_data: str) -> int:
    if type_data == "ShortText":
        return 0
    if type_data == "OneChoice":
        return 1
    if type_data == "MultipleChoice":
        return 2

In [104]:
data["question_clear"] = data["question"].apply(review_to_wordlist)
data["type_encoded"] = data["type"].apply(type_to_encode)

In [105]:
data.head()

,question,type,status,question_clear,type_encoded
0,Отчетство,ShortText,1,[отчетство],0
1,Сколько ву вас сотрудников?,ShortText,0,"[сколько, ву, сотрудник]",0
2,"Часто ли вы планируете, как будете себя вести ...",OneChoice,0,"[часто, планировать, быть, вести, встреча, бес...",1
3,Какого литературного героя вы хотели бы встрет...,ShortText,0,"[какой, литературный, герой, хотеть, встретить...",0
4,Вам легко найти общий язык с незнакомыми людьми?,OneChoice,0,"[легко, найти, общий, язык, незнакомый, человек]",1


In [113]:
from gensim.models import word2vec
model = word2vec.Word2Vec(data['question_clear'], size=300, window=3, workers=4)
#создадим словарь со словами и соответсвующими им векторами
w2v = dict(zip(model.wv.index2word, model.wv.vectors))

In [115]:
model.save('w2v')

In [ ]:
 from gensim.models import word2vec
 model = word2vec.Word2Vec.load('w2v')
 w2v = dict(zip(model.wv.index2word, model.wv.vectors))
 w2v

In [28]:
class mean_vectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(w2v.values())))

    def fit(self, X):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [107]:
data_mean=mean_vectorizer(w2v).fit(data['question_clear']).transform(data['question_clear'])
data_mean = np.concatenate((np.array([[i] for i in data['type_encoded']]), data_mean), axis=1)
data_mean.shape

(366, 301)

In [117]:
data_0_mean=mean_vectorizer(w2v).fit([data['question_clear'][1]]).transform([data['question_clear'][1]])
data_0_mean = np.concatenate((np.array([[data['type_encoded'][1]]]), data_0_mean), axis=1)
data_0_mean.shape

(1, 301)

In [120]:
estimator.predict(data_0_mean)[0][0]

1/1 - 0s


0

In [108]:
def split(train,y,ratio):
    idx = round(train.shape[0] * ratio)
    return train[:idx, :], train[idx:, :], y[:idx], y[idx:]
y = data['status']
Xtr, Xval, ytr, yval = split(data_mean, y,0.8)
Xtr.shape,Xval.shape,ytr.mean(),yval.mean()

((293, 301), (73, 301), 0.3651877133105802, 0.2465753424657534)

In [109]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def baseline_model():
    model = Sequential()
    model.add(Dense(1024, input_dim=Xtr.shape[1], activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1_m, precision_m, recall_m])
    return model

estimator = KerasClassifier(build_fn=baseline_model,epochs=50, nb_epoch=40, batch_size=64,validation_data=(Xval, yval), verbose=2)
estimator.fit(Xtr, ytr)

Epoch 1/50
5/5 - 0s - loss: 0.6930 - accuracy: 0.5563 - f1_m: 0.1172 - precision_m: 0.1167 - recall_m: 0.1795 - val_loss: 0.6895 - val_accuracy: 0.7534 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/50
5/5 - 0s - loss: 0.6871 - accuracy: 0.6348 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 0.6606 - val_accuracy: 0.7534 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 3/50
5/5 - 0s - loss: 0.6531 - accuracy: 0.6348 - f1_m: 0.0174 - precision_m: 0.2000 - recall_m: 0.0091 - val_loss: 0.5069 - val_accuracy: 0.7534 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 4/50
5/5 - 0s - loss: 0.5748 - accuracy: 0.6348 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 0.4139 - val_accuracy: 0.7534 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 5/50
5/5 - 0s - loss: 0.5511 - accuracy: 0

In [92]:
estimator.model.save('model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model/assets


In [99]:
from tensorflow import keras
model = keras.models.load_model('/content/model/', custom_objects={"f1_m": f1_m, 
                                                                   "precision_m": precision_m, 
                                                                   "recall_m": recall_m})

In [ ]:
from sklearn.metrics import mean_squared_error
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pymorphy2
import pandas as pd
import pickle
import jsonlines
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.metrics.pairwise import linear_kernel
from gensim.models import word2vec
from tensorflow import keras
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

morph = pymorphy2.MorphAnalyzer()

stops = set(stopwords.words("english")) | set(stopwords.words("russian"))
def review_to_wordlist(review: str) -> list:
    #1)
    review_text = re.sub("[^а-яА-Яa-zA-Z1-9]"," ", review)
    #2)
    words = review_text.lower().split()
    #3)
    words = [w for w in words if not w in stops]
    #4)
    words = [morph.parse(w)[0].normal_form for w in words ]
    return(words)

def type_to_encode(type_data: str) -> int:
    if type_data == "ShortText":
        return 0
    if type_data == "OneChoice":
        return 1
    if type_data == "MultipleChoice":
        return 2

class mean_vectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(w2v.values())))

    def fit(self, X):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

def check_is_private(question: str, question_type: str) -> int:
    encoded_text = review_to_wordlist(question)
    encoded_type = type_to_encode(question_type)

    model = word2vec.Word2Vec.load('w2v')
    w2v = dict(zip(model.wv.index2word, model.wv.vectors))

    estimator = keras.models.load_model('model', 
                                        custom_objects={"f1_m": f1_m, 
                                                        "precision_m": precision_m, 
                                                        "recall_m": recall_m})
    
    data_mean = mean_vectorizer(w2v).fit([encoded_text]).transform([encoded_text])
    data_mean = np.concatenate((np.array([[encoded_type]]), data_mean), axis=1)

    return estimator.predict(data_mean)[0][0]
